Generate both a CaSR climatological mean map and one from database winds.

Interpolation is carried out on both datasets to a common grid. 
Note: it is not recommended to use this tool over all of Canada - it takes too long and the wind interpolation especially from the observed winds will result in discrepancies. 

There are several predefined averaging areas that can be used. Or you can draw your own using a built in map function.

Liam.Buchart@nrca-rncan.gc.ca
October 6, 2025

In [ ]:
# start with a widget to choose whether we are comparing stations data or just a point location on a map
from ipywidgets import Output
from IPython.display import display
from scipy.interpolate import griddata

import numpy as np
import pandas as pd
import xarray as xr
import ipywidgets as widgets
import os

import json

# load the json
with open("regions_bbox_canada.json", 'r') as f:
    regions_data = json.load(f)

region_names = [region["name"] for region in regions_data["regions"]]

In [15]:
# two selectors for the month and year
Averaging_Select = widgets.Dropdown(
    options=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "DJF", "MAM", "JJA", "SON"], 
    value="8",
    description='Period:',
    )

Year_Select = widgets.Dropdown(
    options=np.arange(2019, 2025+1),
    value=2025,
    description='Year:',
    )

Region_Select = widgets.Dropdown(
    options=region_names,
    value="Prairies",
    description="Region:"
)

display(Averaging_Select, Year_Select, Region_Select)

Dropdown(description='Period:', index=7, options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12…

Dropdown(description='Year:', index=6, options=(np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)…

Dropdown(description='Region:', index=1, options=('British Columbia', 'Prairies', 'Ontario', 'Quebec', 'Atlant…

In [ ]:
# define the grid depending on the Region_Select dropdown
selected_region = Region_Select.value
bounds = next(region["bbox"] for region in regions_data["regions"] if region["name"] == selected_region)

min_lon = bounds["west"]
max_lon = bounds["east"]

min_lat = bounds["south"]
max_lat = bounds["north"]

xi = np.arange(min_lon, max_lon, 0.1)
yi = np.arange(min_lat, max_lat, 0.1)

xi, yi = np.meshgrid(xi, yi)

print("Comparison Grid is Ready...")

In [ ]:
# climatological CaSR grid which is saved in 
def sort_files(dir, Astr):
    all_files = os.listdir(dir)

    day_files = [file for file in all_files if f"{str(Astr)}.h5" in file]

    return day_files

# hacky means of determining if this is a seasonal or monthly average
if Averaging_Select.value.isalpha():
    data_dir = "../climatology/seasonal/"
    p_file = sort_files(data_dir, Averaging_Select.value)

    # load the json
    with open("regions_bbox_canada.json", 'r') as f:
        season_data = json.load(f)

    months = next(season["months"] for season in season_data["seasons"] if season["name"] == Averaging_Select.value)
    mstart = months["mstart"]
    mend = months["mend"]

    sfile = f"1990-2020_seasonal_windspeed_{Averaging_Select.value}.nc"
    dfile = f"1990-2020_seasonal_winddir_{Averaging_Select.value}.nc"
    
elif Averaging_Select.value.isdigit():
    data_dir = "../climatology/monthly"
    mstr = f"{Averaging_Select.value:02d}"
    p_file = sort_files(data_dir, mstr)

    # save some query variables
    mstart = mstr
    mend = mstr

    sfile = f"1990-2020_monthly_windspeed_{str(Averaging_Select.value):02d}.nc"
    dfile = f"1990-2020_monthly_winddir_{str(Averaging_Select.value):02d}.nc"

else:
    data_dir = "Something has gone wrong"

# extract the monthly or seasonal file
casr_speed = xr.open_dataarray(f"{data_dir}/{sfile}")
casr_dir = xr.open_dataarray(f"{data_dir}/{dfile}")


Seasonal


In [ ]:
# query the database for all station data over our averaging period
from etl_station_data import db_query, set_areal_query, last_day_of_month
query = set_areal_query(mstart, mend, Year_Select.value, bounds)
print(f"SQL Query: {query}")
query_file = f"{Averaging_Select.value}_{Region_Select.value}_query_output.csv"

# execute the query on dagan
db_query(query, csv_output=f"./output/{query_file}")

# open the queried file using pandas
obs_winds = pd.read_csv(f"./output/{query_file}")

def regrid_data(xo, yo, zo, xi, yi, method):
    # xo, yo, zo = grids of input data
    # xi, yi = grid to interpolate onto
    # method = string (linar, bilinear)
    return griddata((xo, yo), zo, (xi, yi), method=method)

# interpolate these winds onto the previously created grid

wsi = regrid_data((obs_winds["lon"], obs_winds["lat"]),
                  obs_winds("ws"),
                  (xi, yi), 
                  method="bilinear")

wdi = regrid_data((obs_winds["lon"], obs_winds["lat"]),
                  obs_winds("wdir"),
                  (xi, yi), 
                  method="bilinear")


In [ ]:
# carry out a similar process on the casr data that has been extracted.
with open("./utils/variables.json", 'r') as f:
    casr_vars = json.load(f)

casr_wsi = regrid_data((casr_speed["lons"], casr_speed["lats"]), 
                       casr_speed["mean"],
                       (xi, yi),
                       method="bilinear")

casr_wdi = regrid_data((casr_speed["lons"], casr_speed["lats"]), 
                       casr_speed["mean"],
                       (xi, yi),
                       method="bilinear")


ValueError: Unknown format code 'd' for object of type 'str'

In [ ]:
# begin making plots of the values themselves
import plotly.express as px

# Example data
latitudes = yi
longitudes = xi 

fig = px.scatter_mapbox(
    lat=latitudes,
    lon=longitudes,
    color=wsi,
    color_continuous_scale="Viridis",
    size=wsi,
    size_max=15,
    zoom=3,
    mapbox_style="carto-positron",
    title="WSI Map"
)

fig.show()

